In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import pandas as pd
import torch

from fairseq_signals.utils.store import MemmapReader

/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
root = '/home/aa2650/datasets/code_15/subset'
experiment_root = '/home/aa2650/datasets/code_15/experiments/subset'
fairseq_signals_root = '/home/aa2650/playground/fairseq-signals'
fairseq_signals_root = fairseq_signals_root.rstrip('/')
fairseq_signals_root

'/home/aa2650/playground/fairseq-signals'

In [4]:
segmented_path = f'/home/aa2650/datasets/code_15/subset/test_segmented_split.csv'
segmented_split = pd.read_csv(segmented_path,
    index_col='idx',
)

In [5]:
run = "1"

In [6]:
# The bottom part only needs to be run once
segmented_path = f'/home/aa2650/datasets/code_15/subset/test_segmented_split.csv'
segmented_split = pd.read_csv(segmented_path,
    index_col='idx',
)
# segmented_split['path'] = ('/home/aa2650/datasets/code_15/segmented/') + segmented_split['path']
# segmented_split.to_csv(os.path.join('/home/aa2650/datasets/code_15/segmented_split.csv'))

In [7]:
each_experiment_path = os.path.join(experiment_root, "raw", run)
os.makedirs(each_experiment_path, exist_ok=True)

model_path = f"/home/aa2650/playground/ECG-FM/experiments/raw/checkpoint1.pt"
print(each_experiment_path)


/home/aa2650/datasets/code_15/experiments/subset/raw/1


In [8]:
inference_cmd = f"""fairseq-hydra-inference \\
    task.data="/home/aa2650/datasets/code_15/subset/manifests" \\
    common_eval.path="{model_path}" \\
    common_eval.results_path="{each_experiment_path}" \\
    model.num_labels=6 \\
    dataset.valid_subset="test" \\
    dataset.batch_size=10 \\
    dataset.num_workers=3 \\
    dataset.disable_validation=false \\
    distributed_training.distributed_world_size=1 \\
    distributed_training.find_unused_parameters=True \\
    --config-dir "/home/aa2650/playground/ECG-FM/ckpts/" \\
    --config-name physionet_finetuned
"""

# os.system(inference_cmd)

In [9]:
assert os.path.isfile(f"{each_experiment_path}/outputs_test.npy")
assert os.path.isfile(f"{each_experiment_path}/outputs_test_header.pkl")

In [10]:
# os.path.join('/home/aa2650/playground/ECG-FM/data/code_15/labels/label_def.csv'),

code15_label_def = pd.read_csv(
    os.path.join('/home/aa2650/datasets/code_15/subset/label_def.csv'),
     index_col='name',
)
code15_label_names = code15_label_def.index
code15_label_names

Index(['RBBB', 'LBBB', 'SB', 'ST', 'AF', 'normal_ecg'], dtype='object', name='name')

In [11]:
# Load the array of computed logits
logits = MemmapReader.from_header(f"{each_experiment_path}/outputs_test.npy")[:]
logits.shape

(5873, 6)

In [12]:
# Construct predictions from logits
pred = pd.DataFrame(
    torch.sigmoid(torch.tensor(logits)).numpy(),
    columns=code15_label_names,
)

pred = segmented_split.reset_index().join(pred, how='left').set_index('idx')
pred

,save_file,split,path,sample_size,RBBB,LBBB,SB,ST,AF,normal_ecg
idx,,,,,,,,,,
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.326391e-10,1.015462e-06,4.422764e-02,9.999992e-01,1.327617e-08,7.603068e-10
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.117568e-11,2.404533e-06,9.124491e-02,1.000000e+00,2.867312e-08,2.639516e-09
29077,code_15_100123.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,8.299149e-09,6.740423e-02,3.858091e-01,9.421425e-12,9.991314e-01,1.000000e+00
12710,code_15_1001938.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,7.874606e-02,1.000000e+00,1.000000e+00,9.999996e-01,8.351117e-01,1.000000e+00
35541,code_15_1002557.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,9.850215e-01,9.995652e-01,9.427966e-01,4.880091e-03,4.389795e-06,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.000000e+00,8.379372e-14,3.986274e-01,5.287459e-09,1.000000e+00,1.148103e-03
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.000000e+00,9.199176e-12,2.451412e-02,3.816763e-08,1.000000e+00,3.013300e-03
37132,code_15_998911.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.000000e+00,3.376948e-11,3.203080e-07,9.999999e-01,9.945581e-01,1.000000e+00


In [34]:
# Perform a (crude) thresholding of 0.5 for all labels
pred_thresh = pred.copy()
pred_thresh[code15_label_names] = pred_thresh[code15_label_names] > 0.5

# Construct a readable column of predicted labels for each sample
pred_thresh['labels'] = pred_thresh[code15_label_names].apply(
    lambda row: ', '.join(row.index[row]),
    axis=1,
)
pred_thresh['labels']

idx
4329                               ST
4329                               ST
29077                  AF, normal_ecg
12710    LBBB, SB, ST, AF, normal_ecg
35541      RBBB, LBBB, SB, normal_ecg
                     ...             
15687                        RBBB, AF
15687                        RBBB, AF
37132        RBBB, ST, AF, normal_ecg
418              RBBB, SB, normal_ecg
37915              ST, AF, normal_ecg
Name: labels, Length: 5873, dtype: object

In [14]:
# os.path.join('/home/aa2650/playground/ECG-FM/data/code_15/labels/label_def.csv'),

code15_label_def = pd.read_csv(
    os.path.join('/home/aa2650/datasets/code_15/subset/label_def.csv'),
     index_col='name',
)
code15_label_names = code15_label_def.index
code15_label_names

Index(['RBBB', 'LBBB', 'SB', 'ST', 'AF', 'normal_ecg'], dtype='object', name='name')

In [15]:
code_15_label_def = pd.read_csv("/home/aa2650/playground/ECG-FM/data/code_15/labels/label_def.csv",
     index_col='name',
)
code_15_label_names = code_15_label_def.index
code_15_label_def

,pos_count_all,pos_percent_all
name,,
is_male,138528,0.402691
1dAVb,5699,0.016567
RBBB,9652,0.028058
LBBB,6011,0.017474
SB,5588,0.016244
ST,7571,0.022008
AF,7008,0.020372
normal_ecg,134497,0.390973


In [16]:
label_mapping = {
    'RBBB': 'RBBB',
    'LBBB': 'LBBB',
    'SB': 'SB',
    'ST': 'ST',
    'AF': 'AF',
    'normal_ecg': 'normal_ecg'
}

code15_label_def['name_mapped'] = code15_label_def.index.map(label_mapping)
code15_label_def

,pos_count_all,pos_percent_all,name_mapped
name,,,
RBBB,9672,0.224518,RBBB
LBBB,6026,0.139883,LBBB
SB,5605,0.130110,SB
ST,7584,0.176049,ST
AF,7033,0.163258,AF
normal_ecg,9500,0.220525,normal_ecg


In [17]:
pred_mapped = pred.copy()
pred_mapped.drop(set(code15_label_names) - set(label_mapping.keys()), axis=1, inplace=True)
pred_mapped.rename(label_mapping, axis=1, inplace=True)
pred_mapped

,save_file,split,path,sample_size,RBBB,LBBB,SB,ST,AF,normal_ecg
idx,,,,,,,,,,
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.326391e-10,1.015462e-06,4.422764e-02,9.999992e-01,1.327617e-08,7.603068e-10
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.117568e-11,2.404533e-06,9.124491e-02,1.000000e+00,2.867312e-08,2.639516e-09
29077,code_15_100123.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,8.299149e-09,6.740423e-02,3.858091e-01,9.421425e-12,9.991314e-01,1.000000e+00
12710,code_15_1001938.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,7.874606e-02,1.000000e+00,1.000000e+00,9.999996e-01,8.351117e-01,1.000000e+00
35541,code_15_1002557.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,9.850215e-01,9.995652e-01,9.427966e-01,4.880091e-03,4.389795e-06,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.000000e+00,8.379372e-14,3.986274e-01,5.287459e-09,1.000000e+00,1.148103e-03
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.000000e+00,9.199176e-12,2.451412e-02,3.816763e-08,1.000000e+00,3.013300e-03
37132,code_15_998911.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,1.000000e+00,3.376948e-11,3.203080e-07,9.999999e-01,9.945581e-01,1.000000e+00


In [18]:
pred_thresh_mapped = pred_thresh.copy()
pred_thresh_mapped.drop(set(code15_label_names) - set(label_mapping.keys()), axis=1, inplace=True)
pred_thresh_mapped.rename(label_mapping, axis=1, inplace=True)
pred_thresh_mapped['predicted'] = pred_thresh_mapped[label_mapping.values()].apply(
    lambda row: ', '.join(row.index[row]),
    axis=1,
)
pred_thresh_mapped

,save_file,split,path,sample_size,RBBB,LBBB,SB,ST,AF,normal_ecg,labels,predicted
idx,,,,,,,,,,,,
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,True,False,False,ST,ST
4329,code_15_1000730.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,True,False,False,ST,ST
29077,code_15_100123.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,False,False,False,True,True,"AF, normal_ecg","AF, normal_ecg"
12710,code_15_1001938.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,False,True,True,True,True,True,"LBBB, SB, ST, AF, normal_ecg","LBBB, SB, ST, AF, normal_ecg"
35541,code_15_1002557.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,True,True,True,False,False,True,"RBBB, LBBB, SB, normal_ecg","RBBB, LBBB, SB, normal_ecg"
...,...,...,...,...,...,...,...,...,...,...,...,...
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,True,False,False,False,True,False,"RBBB, AF","RBBB, AF"
15687,code_15_997787.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,True,False,False,False,True,False,"RBBB, AF","RBBB, AF"
37132,code_15_998911.mat,test,/home/aa2650/datasets/code_15/subset/segmented...,2500,True,False,False,True,True,True,"RBBB, ST, AF, normal_ecg","RBBB, ST, AF, normal_ecg"


In [19]:
true_labels = pd.read_csv(os.path.join('/home/aa2650/datasets/code_15/subset/ground_truth_test_labels.csv'), index_col='idx')
true_labels['actual'] = true_labels[label_mapping.values()].apply(
    lambda row: ', '.join(row.index[row]),
    axis=1,
)
true_labels

,RBBB,LBBB,SB,ST,AF,normal_ecg,actual
idx,,,,,,,
37252,False,False,False,False,False,True,normal_ecg
37252,False,False,False,False,False,True,normal_ecg
22756,False,True,False,False,False,False,LBBB
22756,False,True,False,False,False,False,LBBB
2342,True,False,False,False,True,False,"RBBB, AF"
...,...,...,...,...,...,...,...
418,True,False,False,False,False,False,RBBB
12704,True,False,False,False,False,False,RBBB
32348,False,True,False,False,False,False,LBBB


In [20]:
# Visualize predicted and actual labels side-by-side
pred_thresh_mapped[['predicted']].join(true_labels[['actual']], how='left')

# Calculate accuracy
# Compare praedicted and actual labels
comparison = pred_thresh_mapped[['predicted']].join(true_labels[['actual']], how='left')

# Calculate overall accuracy (exact match)
accuracy = (comparison['predicted'] == comparison['actual']).mean()
print(f"Overall accuracy: {accuracy:.2%}")

Overall accuracy: 1.15%


In [24]:
from imblearn.metrics import specificity_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score
)

y_true = comparison['actual']
y_pred = comparison['predicted']

accuracy  = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall    = recall_score(y_true, y_pred, average='weighted')
f1        = f1_score(y_true, y_pred, average='weighted')
specificity = specificity_score(y_true, y_pred, average='weighted')

print(f"Accuracy:    {accuracy:.3f}")
print(f"Precision:   {precision:.3f}")
print(f"Recall:      {recall:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"F1 score:    {f1:.3f}")

Accuracy:    0.012
Precision:   0.057
Recall:      0.012
Specificity: 0.965
F1 score:    0.019


/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer

y_true_str = comparison['actual']
y_pred_str = comparison['predicted']

y_true_list = [labels.split(", ") for labels in y_true_str]
y_pred_list = [labels.split(", ") for labels in y_pred_str]

mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(y_true_list)
y_pred_bin = mlb.transform(y_pred_list)

class_names = mlb.classes_

results = []

for i, cls_name in enumerate(class_names):
    y_true_col = y_true_bin[:, i]
    y_pred_col = y_pred_bin[:, i]
    
    tn, fp, fn, tp = confusion_matrix(y_true_col, y_pred_col).ravel()
    total = tp + tn + fp + fn
    prevalence = (tp + fn) / total
    sensitivity = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
    specificity = tn / (tn + fp)
    npv = tn / (tn + fn)
    accuracy = (tp + tn) / total

    results.append({
        'class': cls_name,
        'prevalence': round(prevalence, 3),
        'f1': round(f1, 3),
        'precision': round(precision, 3),
        'recall': round(sensitivity, 3),
        'specificity': round(specificity, 3),
        'accuracy': round(accuracy, 3)
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)

        class  prevalence     f1  precision  recall  specificity  accuracy
0          AF       0.157  0.178      0.130   0.282        0.650     0.592
1        LBBB       0.151  0.153      0.105   0.279        0.578     0.533
2        RBBB       0.202  0.243      0.170   0.427        0.472     0.463
3          SB       0.134  0.085      0.058   0.161        0.593     0.535
4          ST       0.176  0.243      0.163   0.476        0.480     0.479
5  normal_ecg       0.231  0.382      0.289   0.565        0.582     0.578


/home/aa2650/playground/ECG-FM/virtualenv/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(


In [23]:
with open(f"/home/aa2650/playground/ECG-FM/results/raw/raw-results-{run}.txt", "w") as f:
    f.write(metrics_df.drop(columns=['prevalence']).to_string(index=False))